# Streamlined Extraction of Nucleic Acids and Metabolites from Low- and High-Biomass Samples Using Isopropanol and Matrix Tubes

## 16S analysis

In [ ]:
qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_biom.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_biom.qzv

# There are 381 samples and 11,163 features


In [7]:
# Merge read counts with sample metadata
import pandas as pd

md = pd.read_csv('/matrix/metadata_samples/metadata_samples_qiita_20250218.txt', 
                 sep = '\t')

read_counts_16s_deblur = pd.read_csv('/matrix/metadata_samples/read_counts_16s_deblur.txt', 
                 sep = '\t')

md_with_read_counts_16s_deblur = md.merge(read_counts_16s_deblur,
                                          how = 'left')

md_with_read_counts_16s_deblur.to_csv('/matrix/metadata_samples/metadata_samples_qiita_20250219.txt', 
                 sep = '\t',
                                     index = False)


In [ ]:
# Import FASTA sequences into QIIME 2
qiime tools import \
  --input-path /matrix/data/16S/matrix_16s_deblur_seqs.fasta \
  --output-path /matrix/data/16S/matrix_16s_deblur_seqs.qza \
  --type 'FeatureData[Sequence]'


In [ ]:
# Filter against GreenGenes 2
qiime greengenes2 filter-features \
  --i-feature-table /matrix/data/16S/matrix_16s_deblur_biom.qza \
  --i-reference /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --o-filtered-feature-table /matrix/data/16S/matrix_16s_deblur_gg2_biom.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom.qzv

# There are 381 samples and 8759 features


In [ ]:
# Obtain taxonomy from GreenGenes 2
qiime greengenes2 taxonomy-from-table \
  --i-reference-taxonomy /databases/gg2/2024.09/2024.09.taxonomy.asv.nwk.qza \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom.qza \
  --o-classification /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza


In [ ]:
# Obtain taxonomy from SILVA 138
qiime feature-classifier classify-sklearn \
  --i-classifier /databases/qiime2-2024.10/silva-138-99-515-806-nb-classifier.qza \
  --i-reads /matrix/data/16S/matrix_16s_deblur_seqs.qza \
  --p-n-jobs 2 \
  --o-classification /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza


In [ ]:
# Filter plastid sequences - GreenGenes 2
# NOTE: not sure if it has been updated to include these yet - so check and if not use SILVA 138
qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-exclude mitochondria \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit.qzv

# There are 381 samples and 8759 features

qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-exclude chloroplast \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit_noChl.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit_noChl.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_noMit_noChl.qzv

# There are 381 samples and 8759 features
# NOTE: Confirmed that GreenGenes 2 does not include mitochondria or chloroplast sequences yet


In [ ]:
# Filter plastid sequences - SILVA 138
# NOTE: not sure if GreenGenes 2 has been updated to include these yet - so check and if not use SILVA 138
qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza \
  --p-exclude mitochondria \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit.qzv

# There are 381 samples and 8718 features

qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza \
  --p-exclude chloroplast \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl.qzv

# There are 381 samples and 8619 features


In [ ]:
# Filter whacky sequences

# Unassigned
qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza \
  --p-exclude Unassigned \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned.qzv

# There are 381 samples and 8619 features


# Eukaryota
qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza \
  --p-exclude Eukaryota \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk.qzv

# There are 381 samples and 8619 features


# Bacteria (domain only)
qiime taxa filter-table \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk.qza \
  --i-taxonomy /matrix/data/16S/matrix_16s_deblur_seqs_taxonomy_silva138-515-806.qza \
  --p-exclude d__Bacteria \
  --p-mode 'exact' \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain.qzv

# There are 381 samples and 8598 features


In [ ]:
# Filter controls
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250128.txt \
  --p-where 'empo_1 != "Control"' \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qzv

# There are 311 samples and 8442 features


In [ ]:
# Filter samples that received the metagenomic spike-in
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250305.txt \
  --p-where 'metagenomic_spike_in != "true"' \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_noSpike.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_noSpike.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_noSpike.qzv

# There are 263 samples and 7834 features


In [ ]:
# Split high vs. low biomass samples
## Low biomass = LBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250204.txt \
  --p-where 'plate_biomass = "LBM"' \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm.qzv

# There are 119 samples and 5716 features

## High biomass = HBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250204.txt \
  --p-where 'plate_biomass = "HBM"' \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm.qzv

# There are 144 samples and 2670 features



In [ ]:
# Filter noisy features
## Singletons
### LBM
qiime feature-table filter-features \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm.qza \
  --p-min-frequency 2 \
  --p-min-samples 2 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250314.txt \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qzv

# There are 119 samples and 1742 features

### HBM
qiime feature-table filter-features \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm.qza \
  --p-min-frequency 2 \
  --p-min-samples 2 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qzv

# There are 144 samples and 1740 features


## Reduce feature space for RPCA (features must be present in at least 10% of samples)
### LBM
qiime feature-table filter-features \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza \
  --p-min-frequency 0 \
  --p-min-samples 12 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min10perc.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min10perc.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min10perc.qzv

# There are 118 samples and 232 features
# NOTE - This is a strong filter - try 5% as well

qiime feature-table filter-features \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza \
  --p-min-frequency 0 \
  --p-min-samples 6 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc.qzv

# There are 119 samples and 538 features

### HBM
qiime feature-table filter-features \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qza \
  --p-min-frequency 0 \
  --p-min-samples 14 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc.qzv

# There are 143 samples and 680 features


In [ ]:
# Normalize
# NOTE: for LBM, read counts for EtOH-Matrix, surface-keyboard are 148, 277, 318, 329, 346 - so we need to keep this in mind to retain these samples
## Rarefy
### LBM
qiime feature-table rarefy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza \
  --p-sampling-depth 277 \
  --o-rarefied-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250314.txt \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qzv

# There are 112 samples and 1331 features

### HBM
qiime feature-table rarefy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qza \
  --p-sampling-depth 20636 \
  --o-rarefied-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qzv

# There are 108 samples and 1720 features


## Create matching tables for things robust to rarefaction
### Remove samples only
#### LBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons.qza \
  --p-min-frequency 277 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277.qzv

# There are 112 samples and 1742 features

#### HBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons.qza \
  --p-min-frequency 20636 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636.qzv

# There are 108 samples and 1730 features


### Remove samples and reduce feature space (NOT CURRENTLY USING)
#### LBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc.qza \
  --p-min-frequency 277 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277.qzv

# There are 110 samples and 538 features

#### HBM
qiime feature-table filter-samples \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc.qza \
  --p-min-frequency 20636 \
  --o-filtered-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636.qza

qiime feature-table summarize \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636.qza \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636.qzv

# There are 113 samples and 690 features


In [ ]:
# Alpha-diversity (use rarefied tables)
## LBM
qiime diversity alpha \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --p-metric observed_features \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_alpha_richness.qza

qiime diversity alpha \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --p-metric shannon \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_alpha_shannon.qza

qiime diversity alpha-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric faith_pd \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_alpha_faithspd.qza

## HBM
qiime diversity alpha \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --p-metric observed_features \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_alpha_richness.qza

qiime diversity alpha \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --p-metric shannon \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_alpha_shannon.qza

qiime diversity alpha-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric faith_pd \
  --o-alpha-diversity /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_alpha_faithspd.qza


### NOTE: Currently QIIME 2 alpha-phylogenetic is dropping trailing zeros from sample IDs, so we will unzip the QZA files and manually correct the sample IDs and then use Python to merge those vectors with the sample metadata


In [12]:
import pandas as pd

md = pd.read_csv('/matrix/metadata_samples/metadata_samples_qiita_20250319f.txt',
                 sep = '\t',
                dtype = str)

alpha_faithspd_merged_corrected = pd.read_csv('/matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_merged_noSingletons_rar_alpha_faithspd_corrected.txt',
                                              sep = '\t',
                                             dtype = str)

md_with_alpha = md.merge(alpha_faithspd_merged_corrected,
                         how = 'left')

md_with_alpha.to_csv('/matrix/metadata_samples/metadata_samples_qiita_20250319g.txt',
                 sep = '\t',
                    index = False)


In [ ]:
# Beta-diversity
## Phylo RPCA (use non-rarefied tables)
### WITH 5% (LBM) and 10% (HBM) feature filter
#### LBM
qiime gemelli phylogenetic-rpca-with-taxonomy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --m-taxonomy-file /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_dist.qza \
  --o-counts-by-node-tree /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_tree.qza \
  --o-counts-by-node /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_tree_counts.qza \
  --o-t2t-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_tree_taxonomy.qza

qiime emperor biplot \
  --i-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_biplot.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-feature-metadata-file /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_tree_taxonomy.qza \
  --p-number-of-features 10 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min5perc_min277_phyloRPCA_biplot.qzv

#### HBM
qiime gemelli phylogenetic-rpca-with-taxonomy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --m-taxonomy-file /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_dist.qza \
  --o-counts-by-node-tree /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_tree.qza \
  --o-counts-by-node /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_tree_counts.qza \
  --o-t2t-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_tree_taxonomy.qza

qiime emperor biplot \
  --i-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_biplot.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319a.txt \
  --m-feature-metadata-file /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_tree_taxonomy.qza \
  --p-number-of-features 10 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min10perc_min20636_phyloRPCA_biplot.qzv


### WITHOUT 5% (LBM) and 10% (HBM) feature filter
#### LBM
qiime gemelli phylogenetic-rpca-with-taxonomy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --m-taxonomy-file /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist.qza \
  --o-counts-by-node-tree /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_tree.qza \
  --o-counts-by-node /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_tree_counts.qza \
  --o-t2t-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_tree_taxonomy.qza

qiime emperor biplot \
  --i-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_biplot.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-feature-metadata-file /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_tree_taxonomy.qza \
  --p-number-of-features 10 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_biplot.qzv

#### HBM
qiime gemelli phylogenetic-rpca-with-taxonomy \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --m-taxonomy-file /matrix/data/16S/matrix_16s_deblur_gg2_seqs_taxonomy.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist.qza \
  --o-counts-by-node-tree /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_tree.qza \
  --o-counts-by-node /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_tree_counts.qza \
  --o-t2t-taxonomy /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_tree_taxonomy.qza

qiime emperor biplot \
  --i-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_biplot.qza \
  --m-sample-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319a.txt \
  --m-feature-metadata-file /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_tree_taxonomy.qza \
  --p-number-of-features 10 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_biplot.qzv

# NOTE: Compared ordinations and decided to go without the 5% (LBM) and 10% (HBM) feature filters


In [ ]:
# Beta-diversity
## LBM
### Jaccard
qiime diversity beta \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --p-metric 'jaccard' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard.qza

### UniFrac
qiime diversity beta-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric 'unweighted_unifrac' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac.qza

### weighted UniFrac
qiime diversity beta-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric 'weighted_unifrac' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza

### RPCA
qiime gemelli rpca \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist.qza


## HBM
### Jaccard
qiime diversity beta \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --p-metric 'jaccard' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard.qza

### UniFrac
qiime diversity beta-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric 'unweighted_unifrac' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac.qza

### weighted UniFrac
qiime diversity beta-phylogenetic \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636.qza \
  --i-phylogeny /databases/gg2/2024.09/2024.09.phylogeny.asv.nwk.qza \
  --p-metric 'weighted_unifrac' \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza

### RPCA
qiime gemelli rpca \
  --i-table /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636.qza \
  --p-min-feature-count 0 \
  --p-min-sample-count 0 \
  --p-min-feature-frequency 0 \
  --o-biplot /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_biplot.qza \
  --o-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist.qza


In [ ]:
# Plot
## LBM
### weighted UniFrac
qiime diversity pcoa \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza \
  --o-pcoa /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_pcoa_wunifrac.qza

qiime emperor plot \
  --i-pcoa /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_pcoa_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250309.txt \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_pcoa_wunifrac.qzv

## HBM
### weighted UniFrac
qiime diversity pcoa \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza \
  --o-pcoa /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_pcoa_wunifrac.qza

qiime emperor plot \
  --i-pcoa /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_pcoa_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250309.txt \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_pcoa_wunifrac.qzv


In [ ]:
# Filter distance matrices
## LBM
### Jaccard
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_armpit"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_hand"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_floor"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_keyboard"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard.qza

### Unweighted UniFrac
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_armpit"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_hand"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_floor"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_keyboard"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard.qza

### Weighted UniFrac
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_armpit"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_hand"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_floor"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_keyboard"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard.qza

### RPCA
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_armpit"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_hand"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_floor"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_keyboard"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard.qza

### Phylo-RPCA
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_armpit"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "skin_hand"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_floor"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "surface_swab_keyboard"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard.qza


## HBM
### Jaccard
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "mouse_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_before_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_after_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after.qza

### Unweighted UniFrac
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "mouse_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_before_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_after_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after.qza

### Weighted UniFrac
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "mouse_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_before_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_after_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after.qza

### RPCA
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "mouse_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_before_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_after_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after.qza

### Phylo-RPCA
qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "mouse_feces"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_before_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before.qza

qiime diversity filter-distance-matrix \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-where 'sample_type2 == "human_saliva_after_brushing"' \
  --o-filtered-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after.qza


In [ ]:
# PERMDISP - storage solution
## LBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard_permdisp_storage_solution.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard_permdisp_storage_solution.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard_permdisp_storage_solution.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard_permdisp_storage_solution.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard_permdisp_storage_solution.qzv

## HBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after_permdisp_storage_solution.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after_permdisp_storage_solution.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after_permdisp_storage_solution.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after_permdisp_storage_solution.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before_permdisp_storage_solution.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'storage_solution' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after_permdisp_storage_solution.qzv


In [ ]:
# PERMDISP - extraction method
## LBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard_permdisp_extraction_protocol.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard_permdisp_extraction_protocol.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard_permdisp_extraction_protocol.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard_permdisp_extraction_protocol.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard_permdisp_extraction_protocol.qzv

## HBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after_permdisp_extraction_protocol.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after_permdisp_extraction_protocol.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after_permdisp_extraction_protocol.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after_permdisp_extraction_protocol.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before_permdisp_extraction_protocol.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'extraction_protocol' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after_permdisp_extraction_protocol.qzv


In [ ]:
# PERMDISP - host_subject_id
## LBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard_permdisp_host_subject_id.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard_permdisp_host_subject_id.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard_permdisp_host_subject_id.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard_permdisp_host_subject_id.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard_permdisp_host_subject_id.qzv

## HBM
### Jaccard
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after_permdisp_host_subject_id.qzv

### Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after_permdisp_host_subject_id.qzv

### Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after_permdisp_host_subject_id.qzv

### RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after_permdisp_host_subject_id.qzv

### Phylo-RPCA
qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before_permdisp_host_subject_id.qzv

qiime diversity beta-group-significance \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --m-metadata-column 'host_subject_id' \
  --p-method 'permdisp' \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after_permdisp_host_subject_id.qzv


In [ ]:
# PERMANOVA
## LBM
### Jaccard
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_armpit_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_skin_hand_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_floor_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_jaccard_st2_surface_keyboard_permanova.qzv

### Unweighted UniFrac
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_armpit_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_skin_hand_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_floor_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_unifrac_st2_surface_keyboard_permanova.qzv

### Weighted UniFrac
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_armpit_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_skin_hand_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_floor_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_rar277_dist_wunifrac_st2_surface_keyboard_permanova.qzv

### RPCA
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_armpit_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_skin_hand_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_floor_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_rpca_dist_st2_surface_keyboard_permanova.qzv

### Phylo-RPCA
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_armpit_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_skin_hand_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_floor_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_lbm_noSingletons_min277_phyloRPCA_dist_st2_surface_keyboard_permanova.qzv

## HBM
### Jaccard
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_human_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_stool_mouse_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_before_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_jaccard_st2_saliva_after_permanova.qzv

### Unweighted UniFrac
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_human_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_stool_mouse_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_before_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_unifrac_st2_saliva_after_permanova.qzv

### Weighted UniFrac
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_human_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_stool_mouse_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_before_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_rar20636_dist_wunifrac_st2_saliva_after_permanova.qzv

### RPCA
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_human_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_stool_mouse_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_before_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_rpca_dist_st2_saliva_after_permanova.qzv

### Phylo-RPCA
qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_human_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_stool_mouse_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_before_permanova.qzv

qiime diversity adonis \
  --i-distance-matrix /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after.qza \
  --m-metadata-file /matrix/metadata_samples/metadata_samples_qiita_20250319i.txt \
  --p-formula 'host_subject_id+storage_solution*extraction_protocol' \
  --p-n-jobs 4 \
  --o-visualization /matrix/data/16S/matrix_16s_deblur_gg2_biom_silva_noMit_noChl_noUnassigned_noEuk_noDomain_noControls_noSpike_hbm_noSingletons_min20636_phyloRPCA_dist_st2_saliva_after_permanova.qzv
